# Hagrid, Provisioning and Deployment

The goal for `hagrid` is to be a friendly helper in navigating the world of deployment and provisioning the `syft` stack and to take a multitude of complex steps and turn them into a handful of uniform and consistent commands.

`HAGrid` can deploy to a large variety of targets and by leveraging other tools and formats this can be extended. More specifically, `hagrid` can deploy to:
- `docker`
- any ip (`x.x.x.x`)
- `localhost`
- `azure`
- `gcp`
- `aws` (todo)
- `kubernetes` (coming soon)

## Launch to `Docker`